# Benchmarking
Just how much do we lose by training models on CPU with a spark cluster vs a local GPU?

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import os

Num GPUs Available:  1


In [2]:
from tensorflow.keras.applications.resnet50 import preprocess_input

# Setup training and test split

In [3]:
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
r50_train = tf.keras.preprocessing.image_dataset_from_directory(
    'C:\BioNerds\organized_images', labels='inferred', label_mode='int',
    class_names=None, color_mode='rgb', batch_size=32, image_size=(224,
   224), shuffle=True, seed=314, validation_split=.2, subset='training',
    interpolation='bilinear', follow_links=False, smart_resize=True
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.


In [5]:
r50_test = tf.keras.preprocessing.image_dataset_from_directory(
    'C:\BioNerds\organized_images', labels='inferred', label_mode='int',
    class_names=None, color_mode='rgb', batch_size=32, image_size=(224,
   224), shuffle=True, seed=314, validation_split=.2, subset='validation',
    interpolation='bilinear', follow_links=False, smart_resize=True
)

Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [6]:

r50_test = r50_test.prefetch(buffer_size=2000)
r50_train = r50_train.prefetch(buffer_size=8015)
r50_train = r50_train.shuffle(buffer_size=8015, reshuffle_each_iteration=True)

In [7]:
#model.fit(model_train, epochs = 300, callbacks=[model_checkpoint_callback], validation_data = model_test)

# Define resnet and preprocessing
Preprocessing will consider flips and rotations. We will avoid zooms since we want to avoid inadvertently cropping some or all of the lesion. 

In [8]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(.99),
])

In [9]:
topless = tf.keras.applications.resnet50.ResNet50(
    include_top=False, input_tensor=None, weights = None,
    input_shape=None, pooling=None
)

In [10]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [11]:
from tensorflow.keras.applications.resnet50 import preprocess_input

# Build and Compile the model

In [16]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.image.per_image_standardization(inputs)
x = data_augmentation(x)
x = topless(x)
x = global_average_layer(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(7)(x)
#x = tf.keras.layers.Dense(256, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.5)(x)
#x = tf.keras.layers.Dense(256, activation='relu')(x)
#x = tf.keras.layers.Dense(64, activation='relu')(x)
#x = tf.keras.layers.Dense(7, activation='softmax')(x)
topless_mod = tf.keras.Model(inputs, outputs)

In [13]:
#Save the checkpoints in case this model suddenly performs super well or something
checkpoint_filepath = './oh-god-its-the-benchmark-dont-overwrite-stuff-{epoch:04d}.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [18]:
base_learning_rate = 0.001
topless_mod.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Run 5 epochs and time it

In [25]:


%time topless_mod.fit(r50_train, epochs = 5, validation_data = r50_test, callbacks=[model_checkpoint_callback])

Epoch 1/5
251/251 [==============================] - 48s 158ms/step - loss: 5.2969 - accuracy: 0.6714 - val_loss: 5.4478 - val_accuracy: 0.6620
Epoch 2/5
251/251 [==============================] - 43s 153ms/step - loss: 5.2969 - accuracy: 0.6714 - val_loss: 5.4478 - val_accuracy: 0.6620
Epoch 3/5
251/251 [==============================] - 43s 154ms/step - loss: 5.2969 - accuracy: 0.6714 - val_loss: 5.4478 - val_accuracy: 0.6620
Epoch 4/5
251/251 [==============================] - 43s 153ms/step - loss: 5.2969 - accuracy: 0.6714 - val_loss: 5.4478 - val_accuracy: 0.6620
Epoch 5/5
251/251 [==============================] - 43s 153ms/step - loss: 5.2969 - accuracy: 0.6714 - val_loss: 5.4478 - val_accuracy: 0.6620
Wall time: 3min 38s


In [26]:
print("Now that wasn't so bad")

Now that wasn't so bad
